In [1]:
import numpy as np

# Grid World 환경을 정의하는 클래스
class GridWorld:
    def __init__(self, size=(4, 4), start_state=(0, 0), goal_state=(3, 3), pit_state=(1, 3)):
        """
        Grid World 환경을 초기화합니다.
        
        :param size: 격자 크기 (행, 열)
        :param start_state: 시작 상태 (튜플)
        :param goal_state: 목표 상태 (튜플, +1 보상)
        :param pit_state: 함정 상태 (튜플, -10 보상)
        """
        self.size = size
        self.rows, self.cols = size
        self.start_state = start_state
        self.goal_state = goal_state
        self.pit_state = pit_state
        self.current_state = start_state
        
        # 행동 정의: 0:상, 1:하, 2:좌, 3:우
        self.actions = {0: 'Up', 1: 'Down', 2: 'Left', 3: 'Right'}
        self.num_actions = len(self.actions)

        # 상태 인덱스를 (행, 열)에서 1차원 인덱스로 매핑 (0부터 size*size - 1)
        self.state_to_idx = { (r, c): r * self.cols + c for r in range(self.rows) for c in range(self.cols) }
        self.idx_to_state = { idx: state for state, idx in self.state_to_idx.items() }

    def reset(self):
        """
        환경을 초기 상태로 리셋하고 현재 상태를 반환합니다.
        """
        self.current_state = self.start_state
        return self.current_state

    def get_reward(self, state):
        """
        특정 상태에 대한 보상을 반환합니다.
        """
        if state == self.goal_state:
            return 10  # 목표 지점 (큰 양의 보상)
        elif state == self.pit_state:
            return -10 # 함정 (큰 음의 보상)
        else:
            return -1  # 일반 이동 (작은 음의 보상, 빨리 끝내도록 유도)

    def is_terminal(self, state):
        """
        상태가 종료 상태(목표 또는 함정)인지 확인합니다.
        """
        return state == self.goal_state or state == self.pit_state

    def step(self, action):
        """
        주어진 행동을 실행하고 다음 상태, 보상, 종료 여부를 반환합니다.
        (여기서는 단순한 Deterministic 환경을 가정합니다.)
        
        :param action: 선택된 행동 (0, 1, 2, 3)
        :return: (next_state, reward, done)
        """
        if self.is_terminal(self.current_state):
            # 이미 종료된 상태라면 더 이상 움직일 수 없습니다.
            return self.current_state, 0, True

        r, c = self.current_state
        next_r, next_c = r, c

        # 0:상, 1:하, 2:좌, 3:우
        if action == 0:  # Up
            next_r = max(0, r - 1)
        elif action == 1:  # Down
            next_r = min(self.rows - 1, r + 1)
        elif action == 2:  # Left
            next_c = max(0, c - 1)
        elif action == 3:  # Right
            next_c = min(self.cols - 1, c + 1)

        next_state = (next_r, next_c)

        # 벽을 넘으려는 행동은 제자리로 돌아오게 함 (결정적)
        if next_state == self.current_state and (next_r != r or next_c != c):
             # 경계에서 움직이려 할 때 벽에 부딪히는 로직이 위에 구현되어 있음
             pass

        self.current_state = next_state
        
        reward = self.get_reward(next_state)
        done = self.is_terminal(next_state)

        return next_state, reward, done

In [2]:
# 환경 인스턴스 생성
env = GridWorld()

# 에피소드 시작 (에이전트의 한 번의 시도)
print("=== Grid World 시뮬레이션 시작 ===")
print(f"목표 상태: {env.goal_state}, 함정 상태: {env.pit_state}")

current_state = env.reset()
total_reward = 0
done = False
steps = 0

print(f"\nStep 0: 현재 상태: {current_state}, 누적 보상: {total_reward}")

# 최대 10단계 시뮬레이션 (또는 종료 상태에 도달할 때까지)
while not done and steps < 10:
    # 0, 1, 2, 3 중 무작위 행동 선택 (Random Policy)
    action = np.random.randint(env.num_actions) 
    action_name = env.actions[action]

    # 행동 실행 (Step)
    next_state, reward, done = env.step(action)
    total_reward += reward
    steps += 1
    
    # 결과 출력
    print(f"---")
    print(f"Step {steps}: 행동: {action_name} ({action})")
    print(f"  다음 상태: {next_state}")
    print(f"  획득 보상: {reward}")
    print(f"  누적 보상: {total_reward}")
    
    if done:
        print(f"\n✅ 종료 상태 도달! (최종 상태: {next_state})")
        break

if not done:
    print("\n⚠️ 최대 스텝 수(10)에 도달했지만 종료 상태에 도달하지 못했습니다.")

=== Grid World 시뮬레이션 시작 ===
목표 상태: (3, 3), 함정 상태: (1, 3)

Step 0: 현재 상태: (0, 0), 누적 보상: 0
---
Step 1: 행동: Left (2)
  다음 상태: (0, 0)
  획득 보상: -1
  누적 보상: -1
---
Step 2: 행동: Right (3)
  다음 상태: (0, 1)
  획득 보상: -1
  누적 보상: -2
---
Step 3: 행동: Right (3)
  다음 상태: (0, 2)
  획득 보상: -1
  누적 보상: -3
---
Step 4: 행동: Right (3)
  다음 상태: (0, 3)
  획득 보상: -1
  누적 보상: -4
---
Step 5: 행동: Up (0)
  다음 상태: (0, 3)
  획득 보상: -1
  누적 보상: -5
---
Step 6: 행동: Left (2)
  다음 상태: (0, 2)
  획득 보상: -1
  누적 보상: -6
---
Step 7: 행동: Down (1)
  다음 상태: (1, 2)
  획득 보상: -1
  누적 보상: -7
---
Step 8: 행동: Down (1)
  다음 상태: (2, 2)
  획득 보상: -1
  누적 보상: -8
---
Step 9: 행동: Right (3)
  다음 상태: (2, 3)
  획득 보상: -1
  누적 보상: -9
---
Step 10: 행동: Up (0)
  다음 상태: (1, 3)
  획득 보상: -10
  누적 보상: -19

✅ 종료 상태 도달! (최종 상태: (1, 3))
